# Preambule

### Import and parameters initialization

In [ ]:
using CSV, DataFrames, StatsBase, Plotly, LightGraphs, GraphIO, Distributions

In [ ]:
global const ALPHA = 0.01
global const NBSHUFFLE = 1000000

In [ ]:
srand(1)

### Modified export functions

In [ ]:
"""
Modified from GraphIO.jl
Write a graph `g` with node labels `nlabs` given in a dictionary to an IO stream `io` in the
[GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. Return 1.
"""
function saveLabeledGml(io::IO, g::LightGraphs.AbstractGraph, nlabs::Dict{Int64,String})
    println(io, "graph")
    println(io, "[")
    is_directed(g) && println(io, "directed 1")
    for i = 1:nv(g)
        println(io, "\tnode")
        println(io, "\t[")
        println(io, "\t\tid $i")
        println(io, "\t\tlabel \"", nlabs[i], '"')
        println(io, "\t]")
    end
    for e in LightGraphs.edges(g)
        s, t = Tuple(e)
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t]")
    end
    println(io, "]")
    return 1
end

"""
Modified from GraphIO.jl
Write a graph `g` with node labels `nlabs` and edge labels
'elabs' given in two dictionaries to an IO stream `io` in the
[GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. Return 1.
"""
function saveLabeledGml(io::IO, g::LightGraphs.AbstractGraph, nlabs::Dict{Int64,String},
    elabs::Dict{Tuple,String})
    println(io, "graph")
    println(io, "[")
    is_directed(g) && println(io, "directed 1")
    for i = 1:nv(g)
        println(io, "\tnode")
        println(io, "\t[")
        println(io, "\t\tid $i")
        println(io, "\t\tlabel \"", nlabs[i], '"')
        println(io, "\t]")
    end
    for e in LightGraphs.edges(g)
        s, t = Tuple(e)
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t\tlabel \"", elabs[(s,t)], '"')
        println(io, "\t]")
    end
    println(io, "]")
    return 1
end

"""
Modified from GraphIO.jl
Write a graph `g` with node labels `nlabs` and node class
'nclass' given in two dictionaries to an IO stream `io` in the
[GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. Return 1.
"""
function saveLabeledGml(io::IO, g::LightGraphs.AbstractGraph, nlabs::Dict{Int64,String},
    elabs::Dict{Int64,Int64})
    println(io, "graph")
    println(io, "[")
    is_directed(g) && println(io, "directed 1")
    for i = 1:nv(g)
        println(io, "\tnode")
        println(io, "\t[")
        println(io, "\t\tid $i")
        println(io, "\t\tlabel \"", nlabs[i], '"')
        println(io, "\t\tclass ", elabs[i])
        println(io, "\t]")
    end
    for e in LightGraphs.edges(g)
        s, t = Tuple(e)
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t]")
    end
    println(io, "]")
    return 1
end

"""
Modified from GraphIO.jl
Write a graph `g` with node labels `nlabs` and 2 sets of edge labels
'elabs' given in three dictionaries to an IO stream `io` in the
[GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. Return 1.
"""
function saveLabeledGml(io::IO, g::LightGraphs.AbstractGraph, nlabs::Dict{Int64,String},
    elabs1::Dict{Tuple{Int64,Int64},Float64}, elabs2::Dict{Tuple{Int64,Int64},Float64})
    println(io, "graph")
    println(io, "[")
    is_directed(g) && println(io, "directed 1")
    for i = 1:nv(g)
        println(io, "\tnode")
        println(io, "\t[")
        println(io, "\t\tid $i")
        println(io, "\t\tlabel \"", nlabs[i], '"')
        println(io, "\t]")
    end
    for (e,v) = elabs1
        s, t = e
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t\tweight $v")
        println(io, "\t\tclass 1")
        println(io, "\t]")
    end
    for (e,v) = elabs2
        s, t = e
        println(io, "\tedge")
        println(io, "\t[")
        println(io, "\t\tsource $s")
        println(io, "\t\ttarget $t")
        println(io, "\t\tweight $v")
        println(io, "\t\tclass 2")
        println(io, "\t]")
    end
    println(io, "]")
    return 1
end

# BAF-PBAF complexes structure inference
![Baf structure](BAF_struct.jpg)

* 250A = ARID1A
* 250B = (ARID1B)
* 60A = SMARCD1
* 60B = SMARCD2
* 60C = SMARCD3
* BCL7A = BCL7A
* BCL7B = BCL7B
* BCL7C = -BCL7C
* 155 = SMARCC1 
* 170 = SMARCC2
* 57 = SMARCE1 
* BRG1 = SMARCA4 
* BRM = SMARCA2
* 53A = ACTL6A
* $\beta$-actin = (ACTB)
* SS18 = (SS18)
* 47 = SMARCB1
* 45D = DPF2
* (45B) = DPF1
* (45C) = DPF3
* (SS18L1) = SS18L1

* BRD9 = (BRD9)

In [ ]:
colnames = ["Units"    
 "ACTB"     
 "ARID1B"   
 "ARID2"    
 "BCL11A"   
 "BCL11B"   
 "BCL7A"    
 "BCL7B"    
 "BRD7"     
 "BRD9"     
 "DPF1"     
 "DPF2"     
 "DPF3"     
 "PBRM1"    
 "PHF10"    
 "SMARCA2"  
 "SMARCA4.4"
 "SMARCA4.6"
 "SMARCC1"  
 "SMARCC2"  
 "SMARCD1"  
 "SMARCD2"  
 "SMARCD3"]
aridData = CSV.read("ARID1A-data.csv"; delim='\t', header=colnames, datarow=2)

In [ ]:
foreach(x -> aridData[x] = log2.(aridData[x]), names(aridData[:,2:end]))

In [ ]:
aridPval = CSV.read("ARID1A-pval.csv"; delim='\t', header=colnames, datarow=2)
aridPval[1] = aridData[1]

In [ ]:
colnames = ["Units"    
 "ACTB"     
 "ARID1A.10"
 "ARID1A.3" 
 "ARID1B"   
 "ARID2"    
 "BCL11A"   
 "BCL11B"   
 "BCL7A"    
 "BCL7B"    
 "BRD7"     
 "BRD9"     
 "DPF1"     
 "DPF2"     
 "DPF3"     
 "PBRM1"    
 "PHF10"    
 "SMARCA2"  
 "SMARCC1"  
 "SMARCC2"  
 "SMARCD1"  
 "SMARCD2"  
 "SMARCD3"]
brgData = CSV.read("BRG1-data.csv"; delim='\t', header=colnames, datarow=2)
foreach(x -> brgData[x] = log2.(brgData[x]), names(brgData[:,2:end]))

In [ ]:
brgPval = CSV.read("BRG1-pval.csv"; delim='\t', header=colnames, datarow=2)
brgPval[1] = brgData[1]

We now remove variations where the fold change is not significantly greater than zero.

In [ ]:
for i in 2:length(brgData)
    for j in 1:length(brgData[i])
        if brgPval[j,i] > ALPHA
            brgData[j,i] = 0
        end
    end
end

In [ ]:
for i in 2:length(aridData)
    for j in 1:length(aridData[i])
        # Some values were stored as factors instead of floats, and could not be compared to ALPHA
        try
            if aridPval[j,i] > ALPHA
                aridData[j,i] = 0
            end
        catch e
            if isa(e, MethodError) # In case of type error when comparing the variable to ALPHA 
                if float(string(aridPval[j,i])) > ALPHA # Try converting the faulty variable
                    aridData[j,i] = 0
                end
            end
        end
    end
end

## BAF complex structure
Pulling down ARID1A only captures the BAF complex

In [ ]:
# Join SMARCA4.4 and SMARCA4.6
delete!(aridData, Symbol("SMARCA4.6"))
rename!(aridData, Symbol("SMARCA4.4") => :SMARCA4)

In [ ]:
nbARIDpd = length(studyARIDpd)
nbARIDko = length(studyARIDko)
println(countnz(Matrix(aridData[:,2:end])))
for i = 1:NBSHUFFLE
    # Select two random cells
    x1 = rand(1:nbARIDpd)
    y1 = rand(1:nbARIDko)
    x2 = rand(1:nbARIDpd)
    y2 = rand(1:nbARIDko)
    # Do not change diagonal
    if studyARIDpd[x1] != studyARIDko[y1] && studyARIDpd[x2] != studyARIDko[y2]
        tmp = aridData[x1,y1+1]
        aridData[x1,y1+1] = aridData[x2,y2+1]
        aridData[x2,y2+1] = tmp
    end
end
println(countnz(Matrix(aridData[:,2:end])))

In [ ]:
init_notebook(true)

traceArid = heatmap(
    x=aridData[1],
    y=names(aridData[2:end]),
    z=convert(Array, aridData[:,2:end])
)

#===== Color mapping
We want a linear scale from blue to white (minimal value to zero)
then from white to red (zero to maximal value).
Plotly expect linear scales with endpoints in zero (minimal value)
to 1 (maximal value), therefore we transform the coordinate c in
our scale to plotly's scale p by the following transformation:
p = (c - minVal)/(maxVal - minVal)
=====#
coordZero = -minimum(convert(Array, aridData[:,2:end])) /
    (maximum(convert(Array, aridData[:,2:end])) - minimum(convert(Array, aridData[:,2:end])))
styleArid = Style(global_trace=attr(colorscale=[[0, "rgb(0,0,255)"], [coordZero, "rgb(255,255,255)"], [1, "rgb(255,0,0)"]]))
layoutArid = Layout(;margin_l = 100, margin_t = 20, yaxis_title="<b>Knocked-out gene</b>", xaxis_title = "<b>BAF subunit</b>")

plot(traceArid, layoutArid, style=styleArid)

## BAF-PBAF complex structure
Pulling down SMARCA4 captures both the BAF and PBAF complexes

In [ ]:
# Join ARID1A.10 and ARID1A.3
delete!(brgData, Symbol("ARID1A.3"))
rename!(brgData, Symbol("ARID1A.10") => :ARID1A)

In [ ]:
nbBRGpd = length(studyBRGpd)
nbBRGko = length(studyBRGko)
println(countnz(Matrix(brgData[:,2:end])))
for i = 1:NBSHUFFLE
    # Select two random cells
    x1 = rand(1:nbBRGpd)
    y1 = rand(1:nbBRGko)
    x2 = rand(1:nbBRGpd)
    y2 = rand(1:nbBRGko)
    # Do not change diagonal
    if studyBRGpd[x1] != studyBRGko[y1] && studyBRGpd[x2] != studyBRGko[y2]
        tmp = brgData[x1,y1+1]
        brgData[x1,y1+1] = brgData[x2,y2+1]
        brgData[x2,y2+1] = tmp
    end
end
println(countnz(Matrix(brgData[:,2:end])))

In [ ]:
traceBrg = heatmap(
    x=brgData[1],
    y=names(brgData[2:end]),
    z=convert(Array, brgData[:,2:end])
)

#===== Color mapping
We want a linear scale from blue to white (minimal value to zero)
then from white to red (zero to maximal value).
Plotly expect linear scales with endpoints in zero (minimal value)
to 1 (maximal value), therefore we transform the coordinate c in
our scale to plotly's scale p by the following transformation:
p = (c - minVal)/(maxVal - minVal)
=====#
coordZero = -minimum(convert(Array, brgData[:,2:end])) /
    (maximum(convert(Array, brgData[:,2:end])) - minimum(convert(Array, brgData[:,2:end])))
styleBrg = Style(global_trace=attr(colorscale=[[0, "rgb(0,0,255)"], [coordZero, "rgb(255,255,255)"], [1, "rgb(255,0,0)"]]))
layoutBrg = Layout(;margin_l = 100, margin_t = 20, yaxis_title="<b>Knocked-out gene</b>", xaxis_title = "<b>(P)BAF subunit</b>")

plot(traceBrg, layoutBrg, style=styleBrg)

## Define constants used by the algorithm

In [ ]:
studyARIDko = convert(Array{String,1}, names(aridData[2:end]))
studyARIDpd = convert(Array{String,1}, aridData[1])
studyBRGko = convert(Array{String,1}, names(brgData[2:end]))
studyBRGpd = convert(Array{String,1}, brgData[1])
unitDictString = Dict(enumerate(sort(union(studyARIDko, studyARIDpd, studyBRGko, studyBRGpd))))
unitDict = map(reverse, unitDictString)
unitList = [string(u) for u in keys(unitDict)]
# How many subunits are we considering?
const M = length(unitDict)

In [ ]:
edgeTypesARID = Dict{Float64, Dict{Tuple, String}}()

colnames = ["Units"    
 "ACTB"     
 "ARID1B"   
 "ARID2"    
 "BCL11A"   
 "BCL11B"   
 "BCL7A"    
 "BCL7B"    
 "BRD7"     
 "BRD9"     
 "DPF1"     
 "DPF2"     
 "DPF3"     
 "PBRM1"    
 "PHF10"    
 "SMARCA2"  
 "SMARCA4.4"
 "SMARCA4.6"
 "SMARCC1"  
 "SMARCC2"  
 "SMARCD1"  
 "SMARCD2"  
 "SMARCD3"]

for alpha = [0.1, 0.05, 0.01, 0.005]    
    aridData = CSV.read("ARID1A-data.csv"; delim='\t', header=colnames, datarow=2)
    
    foreach(x -> aridData[x] = log2.(aridData[x]), names(aridData[:,2:end]))

    for i in 2:length(aridData)
        for j in 1:length(aridData[i])
            # Some values were stored as factors instead of floats, and could not be compared to ALPHA
            try
                if aridPval[j,i] > alpha
                    aridData[j,i] = 0
                end
            catch e
                if isa(e, MethodError) # In case of type error when comparing the variable to ALPHA 
                    if float(string(aridPval[j,i])) > alpha # Try converting the faulty variable
                        aridData[j,i] = 0
                    end
                end
            end
        end
    end
    
     # Join SMARCA4.4 and SMARCA4.6
    delete!(aridData, Symbol("SMARCA4.6"))
    rename!(aridData, Symbol("SMARCA4.4") => :SMARCA4)
    
    for i = 1:NBSHUFFLE
        # Select two random cells
        x1 = rand(1:nbARIDpd)
        y1 = rand(1:nbARIDko)
        x2 = rand(1:nbARIDpd)
        y2 = rand(1:nbARIDko)
        # Do not change diagonal
        if studyARIDpd[x1] != studyARIDko[y1] && studyARIDpd[x2] != studyARIDko[y2]
            tmp = aridData[x1,y1+1]
            aridData[x1,y1+1] = aridData[x2,y2+1]
            aridData[x2,y2+1] = tmp
        end
    end
    
    # Store the sign of the log2-fold-change associated with each link
    edgeTypesARID[alpha] = Dict{Tuple, String}()

    # Parse each column
    for x = names(aridData[:,2:end])
        for y = 1:length(aridData[x])
            if aridData[y,x] < 0
                edgeTypesARID[alpha][(unitDict[String(x)], unitDict[String(aridData[y,:Units])])] = "inhibits"
            elseif aridData[y,x] > 0
                edgeTypesARID[alpha][(unitDict[String(x)], unitDict[String(aridData[y,:Units])])] = "enhances"
            end
        end
    end 
end

In [ ]:
edgeTypesBRG = Dict{Float64, Dict{Tuple, String}}()

colnames = ["Units"    
 "ACTB"     
 "ARID1A.10"
 "ARID1A.3" 
 "ARID1B"   
 "ARID2"    
 "BCL11A"   
 "BCL11B"   
 "BCL7A"    
 "BCL7B"    
 "BRD7"     
 "BRD9"     
 "DPF1"     
 "DPF2"     
 "DPF3"     
 "PBRM1"    
 "PHF10"    
 "SMARCA2"  
 "SMARCC1"  
 "SMARCC2"  
 "SMARCD1"  
 "SMARCD2"  
 "SMARCD3"]

for alpha = [0.1, 0.05, 0.01, 0.005]    
    brgData = CSV.read("BRG1-data.csv"; delim='\t', header=colnames, datarow=2)
    
    foreach(x -> brgData[x] = log2.(brgData[x]), names(brgData[:,2:end]))

    for i in 2:length(brgData)
        for j in 1:length(brgData[i])
            # Some values were stored as factors instead of floats, and could not be compared to ALPHA
            try
                if brgPval[j,i] > alpha
                    brgData[j,i] = 0
                end
            catch e
                if isa(e, MethodError) # In case of type error when comparing the variable to ALPHA 
                    if float(string(brgPval[j,i])) > alpha # Try converting the faulty variable
                        brgData[j,i] = 0
                    end
                end
            end
        end
    end
       
    # Join ARID1A.10 and ARID1A.3
    delete!(brgData, Symbol("ARID1A.3"))
    rename!(brgData, Symbol("ARID1A.10") => :ARID1A)
    
        for i = 1:NBSHUFFLE
        # Select two random cells
        x1 = rand(1:nbBRGpd)
        y1 = rand(1:nbBRGko)
        x2 = rand(1:nbBRGpd)
        y2 = rand(1:nbBRGko)
        # Do not change diagonal
        if studyBRGpd[x1] != studyBRGko[y1] && studyBRGpd[x2] != studyBRGko[y2]
            tmp = brgData[x1,y1+1]
            brgData[x1,y1+1] = brgData[x2,y2+1]
            brgData[x2,y2+1] = tmp
        end
    end
    
    # Store the sign of the log2-fold-change associated with each link
    edgeTypesBRG[alpha] = Dict{Tuple, String}()

    # Parse each column
    for x = names(brgData[:,2:end])
        for y = 1:length(brgData[x])
            if brgData[y,x] < 0
                edgeTypesBRG[alpha][(unitDict[String(x)], unitDict[String(brgData[y,:Units])])] = "inhibits"
            elseif brgData[y,x] > 0
                edgeTypesBRG[alpha][(unitDict[String(x)], unitDict[String(brgData[y,:Units])])] = "enhances"
            end
        end
    end
end

In [ ]:
const inhibitEdge = "inhibits"
const enhanceEdge = "enhances"
# ARID1A and SMARCA4 should not be deleted in their respective immunoprecipitations
@assert !("ARID1A" in studyARIDko)
@assert !("SMARCA4" in studyBRGko)
    #=====
    When we delete a node from a lightgraph, the node to
    remove is swapped with the last node in the node list.
    To ensure that the index of ARID1A is stable, we make
    sur that it is never knocked out nor the last node.
    =====#
# Remember ARID1A and SMARCA4 indices
const brgIndex = [i for (i,u) in enumerate(unitList) if u == "SMARCA4"][1]
const aridIndex = [i for (i,u) in enumerate(unitList) if u == "ARID1A"][1]
# ARID1A and SMARCA4 should not be the last subunits in the lists
@assert aridIndex != M
@assert brgIndex != M

In [ ]:
# Avoid to compute these sets in each iteration of the structureToPulldowns function
studyARIDpdIndices = [ipd for (ipd, pd) in unitDictString if pd in studyARIDpd]
studyARIDkoIndices = [iko for (iko, ko) in unitDictString if ko in studyARIDko]
studyBRGpdIndices = [ipd for (ipd, pd) in unitDictString if pd in studyBRGpd]
studyBRGkoIndices = [iko for (iko, ko) in unitDictString if ko in studyBRGko]
studyALLkoIndices = intersect(studyARIDkoIndices, studyBRGkoIndices)
studyALLpdIndices = intersect(studyARIDpdIndices, studyBRGpdIndices)
studyOnlyBRGpdIndices = setdiff(studyBRGpdIndices, studyARIDpdIndices)
studyOnlyARIDpdIndices = setdiff(studyARIDpdIndices, studyBRGpdIndices)
studyOnlyBRGkoIndices = setdiff(studyBRGkoIndices, studyARIDkoIndices)
studyOnlyARIDkoIndices = setdiff(studyARIDkoIndices, studyBRGkoIndices)

## Define graph Julia struct
Pulldown graphs contain the directed graph of activation/inhibition, the node and edges annotations.  
Structure graphs contain the structural graph, the node annotations and the competition classes of each node.

In [ ]:
mutable struct pulldownGraph
    graph::SimpleDiGraph
    nodes::Dict{Int64,String}
    edges::Dict{Tuple, String}
end

In [ ]:
mutable struct structureGraph
    graph::SimpleGraph
    nodes::Dict{Int64,String}
    competition::Dict{Int64,Int64}
end

## Define graph functions

In [ ]:
"""
Compute pulldown graph corresponding to a
structure graph given as argument
"""
function structureToPulldowns(sGraph::structureGraph)
    # Initialise two pulldownGraphs
    # with the studied nodes and no edges
    pGraphARID = pulldownGraph(
        SimpleDiGraph(M),
        sGraph.nodes,
        Dict{Tuple, String}()
    )
    
    pGraphBRG = pulldownGraph(
        SimpleDiGraph(M),
        sGraph.nodes,
        Dict{Tuple, String}()
    )
    
    # Create dict from competitions between units                
    competitionDict = getCompetitionDict(sGraph.competition)
    
    # For units knocked-out in both precipitations
    for iko = studyALLkoIndices
        # Compute what units are still connected to ARID1A and SMARCA4
        pulledARIDComponent = getARIDPulledComponent(sGraph.graph, iko) 
        pulledBRGComponent = getBRGPulledComponent(sGraph.graph, iko) 
        
        # Check what would be observed for each pulled down subunit
        # Starting with the subunits pulled down in both precipitations
        for ipd = studyALLpdIndices
            if ipd == iko
                # The KOed subunit is inhibited
                add_pulldown_edge!(inhibitEdge, pGraphARID, ipd)
                add_pulldown_edge!(inhibitEdge, pGraphBRG, ipd)
            else
                # If the subunit is the last in the node list,
                # its index has been swapped with the deleted node
                if ipd == M
                    if !(iko in pulledARIDComponent)
                        add_pulldown_edge!(inhibitEdge, pGraphARID, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphARID, pulledARIDComponent,
                        competitionDict, ipd, iko)
                    end
                    if !(iko in pulledBRGComponent)
                        add_pulldown_edge!(inhibitEdge, pGraphBRG, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphBRG, pulledBRGComponent,
                        competitionDict, ipd, iko)
                    end                    
                    continue # Look at next pulldowned subunit
                else
                    if !(ipd in pulledARIDComponent)
                        # If a subunit is not in the component connected
                        # to ARID1A, the KO will decrease the quantity of
                        # this subunit that will be pulled-down
                        add_pulldown_edge!(inhibitEdge, pGraphARID, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphARID, pulledARIDComponent,
                            competitionDict, ipd, iko)
                    end
                    if !(ipd in pulledBRGComponent)
                        # If a subunit is not in the component connected
                        # to ARID1A, the KO will decrease the quantity of
                        # this subunit that will be pulled-down
                        add_pulldown_edge!(inhibitEdge, pGraphBRG, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphBRG, pulledBRGComponent,
                            competitionDict, ipd, iko)
                    end
                end
            end
        end 
        
        # Then for ARID1A precipitation
        for ipd = studyOnlyARIDpdIndices
            if ipd == iko
                # The KOed subunit is inhibited
                add_pulldown_edge!(inhibitEdge, pGraphARID, ipd)
            else
                # If the subunit is the last in the node list,
                # its index has been swapped with the deleted node
                if ipd == M
                    if !(iko in pulledARIDComponent)
                        add_pulldown_edge!(inhibitEdge, pGraphARID, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphARID, pulledARIDComponent,
                        competitionDict, ipd, iko)
                    end               
                    continue # Look at next pulldowned subunit
                elseif !(ipd in pulledARIDComponent)
                    # If a subunit is not in the component connected
                    # to ARID1A, the KO will decrease the quantity of
                    # this subunit that will be pulled-down
                    add_pulldown_edge!(inhibitEdge, pGraphARID, iko, ipd)
                else
                    # The PD subunit is connected
                    enhanceIfDisconnectedCompetition!(pGraphARID, pulledARIDComponent,
                        competitionDict, ipd, iko)
                end
            end
        end
        
        # Finally for SMARCA4 precipitation
        for ipd = studyOnlyBRGpdIndices
            if ipd == iko
                # The KOed subunit is inhibited
                add_pulldown_edge!(inhibitEdge, pGraphBRG, ipd)
            else
                # If the subunit is the last in the node list,
                # its index has been swapped with the deleted node
                if ipd == M
                    if !(iko in pulledBRGComponent)
                        add_pulldown_edge!(inhibitEdge, pGraphBRG, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphBRG, pulledBRGComponent,
                        competitionDict, ipd, iko)
                    end               
                    continue # Look at next pulldowned subunit
                elseif !(ipd in pulledBRGComponent)
                    # If a subunit is not in the component connected
                    # to ARID1A, the KO will decrease the quantity of
                    # this subunit that will be pulled-down
                    add_pulldown_edge!(inhibitEdge, pGraphBRG, iko, ipd)
                else
                    # The PD subunit is connected
                    enhanceIfDisconnectedCompetition!(pGraphBRG, pulledBRGComponent,
                        competitionDict, ipd, iko)
                end
            end
        end
    end
    
    # Same for units knocked-out only in ARID1A precipitation
    for iko = studyOnlyARIDkoIndices
        # Compute what units are still connected to SMARCA4
        pulledARIDComponent = getARIDPulledComponent(sGraph.graph, iko) 
        
        # Check what would be observed for each pulled down subunit
        for ipd = studyARIDpdIndices
            if ipd == iko
                # The KOed subunit is inhibited
                add_pulldown_edge!(inhibitEdge, pGraphARID, ipd)
            else
                # If the subunit is the last in the node list,
                # its index has been swapped with the deleted node
                if ipd == M
                    if !(iko in pulledARIDComponent)
                        add_pulldown_edge!(inhibitEdge, pGraphARID, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphARID, pulledARIDComponent,
                        competitionDict, ipd, iko)
                    end                  
                    continue # Look at next pulldowned subunit
                else
                    if !(ipd in pulledARIDComponent)
                        # If a subunit is not in the component connected
                        # to ARID1A, the KO will decrease the quantity of
                        # this subunit that will be pulled-down
                        add_pulldown_edge!(inhibitEdge, pGraphARID, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphARID, pulledARIDComponent,
                            competitionDict, ipd, iko)
                    end
                end
            end
        end 
    end
    
    # Same for units knocked-out only in ARID1A precipitation
    for iko = studyOnlyBRGkoIndices
        # Compute what units are still connected to SMARCA4
        pulledBRGComponent = getBRGPulledComponent(sGraph.graph, iko) 
        
        # Check what would be observed for each pulled down subunit
        for ipd = studyBRGpdIndices
            if ipd == iko
                # The KOed subunit is inhibited
                add_pulldown_edge!(inhibitEdge, pGraphBRG, ipd)
            else
                # If the subunit is the last in the node list,
                # its index has been swapped with the deleted node
                if ipd == M
                    if !(iko in pulledBRGComponent)
                        add_pulldown_edge!(inhibitEdge, pGraphBRG, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphBRG, pulledBRGComponent,
                        competitionDict, ipd, iko)
                    end                  
                    continue # Look at next pulldowned subunit
                else
                    if !(ipd in pulledBRGComponent)
                        # If a subunit is not in the component connected
                        # to ARID1A, the KO will decrease the quantity of
                        # this subunit that will be pulled-down
                        add_pulldown_edge!(inhibitEdge, pGraphBRG, iko, ipd)
                    else
                        # The PD subunit is connected
                        enhanceIfDisconnectedCompetition!(pGraphBRG, pulledBRGComponent,
                            competitionDict, ipd, iko)
                    end
                end
            end
        end 
    end
    
    return(pGraphARID, pGraphBRG)
end

"""
Return a list of all subunits still connected
to ARID1A after a given KO is performed
"""        
function getARIDPulledComponent(graph::LightGraphs.SimpleGraphs.SimpleGraph{Int64}, iko::Int64)
    perturbGraph = copy(graph)
    rem_vertex!(perturbGraph, iko)
    pulledComponent = Array{Int64,1}
    for component in connected_components(perturbGraph) if aridIndex in component
        return(component)
    end end
end

"""
Return a list of all subunits still connected
to SMARCA4 after a given KO is performed
"""        
function getBRGPulledComponent(graph::LightGraphs.SimpleGraphs.SimpleGraph{Int64}, iko::Int64)
    perturbGraph = copy(graph)
    rem_vertex!(perturbGraph, iko)
    pulledComponent = Array{Int64,1}
    for component in connected_components(perturbGraph) if brgIndex in component
        return(component)
    end end
end

In [ ]:
"""
Add a link to a pulldownGraph
"""
function add_pulldown_edge!(edgeType::String, pGraph::pulldownGraph, from::Int64, to = from)
    add_edge!(pGraph.graph, from, to)
    pGraph.edges[(from, to)] = edgeType
end
                        
"""
Create a dictionary associating a subunit with its competitors
"""
function getCompetitionDict(competition::Dict{Int64,Int64})
    competitionDF = DataFrame(Int64, M, 2)
    for i in 1:M
        competitionDF[i,1] = i
        competitionDF[i,2] = competition[i]
    end
    names!(competitionDF, [:Key, :Value])
    
    competitionDict = Dict{Int64, Array}()
    for df in groupby(competitionDF, :Value)
        for value in df[:Key]
            competitionDict[value] = [i for i in df[:Key] if i != value]
        end
    end
    
    return(competitionDict)
end

"""
Predict enrichment if a KO disconnect a competitor
of a subunit
"""
function enhanceIfDisconnectedCompetition!(pGraph::pulldownGraph, 
        pulledComponent::Array{Int64,1}, competitionDict::Dict{Int64, Array},
        ipd::Int64, iko::Int64)
    # For the KOed subunit
    if ipd in competitionDict[iko]
        add_pulldown_edge!(enhanceEdge, pGraph, iko, ipd)
        return(true) # An edge has been added
    end    
    # For all non-KOed subunit
    for inc = (j for j in 1:(M-1) if !(j in pulledComponent))
        if inc == iko
            # If the subunit has the index 'iko' it is
            # actually the last subunit, that has been
            # swapped with the KOed subunit
            inc = M
        end
        if ipd in competitionDict[inc]
            add_pulldown_edge!(enhanceEdge, pGraph, iko, ipd)
            return(true) # An edge has been added
        end
    end
    return(false) # No edge has been added
end
                        
"""
Enforce the connectivity of a structureGraph
"""
function connectGraph!(sGraph::structureGraph)
    while !is_connected(sGraph.graph)
        mutateAddEdge!(sGraph)
end end
                        
"""
Attribute random competition classes for subunits not
yet present in competition dictionary of a structureGraph
"""
function randomCompetitionGraph!(sGraph::structureGraph)
    graph = sGraph.graph
    competition = sGraph.competition
    
    for i = 1:M
        # For all subunits not in the competition dict
        if !(i in keys(competition))
            # Continue until a competition class has been attributed
            while true
                # Assign random competition class
                newComp = rand(1:M)
                if all([competition[n] != newComp for n in intersect(neighbors(graph, i), keys(competition))])
                    competition[i] = rand(1:M)
                    break
                end
                # This competition would link interactors, try again
            end
        end
    end
end

## Define mutation functions

In [ ]:
"""
Mutate a single structure graph
The keywords contain the mutation parameters:
    p_add: add edge probability
    p_del: del edge probability
    p_swp: swap edge probability
    p_cmp: competition class probability
"""
function mutateStructureGraph!(sGraph::structureGraph; 
        p_add = 0.1, p_del = p_add, p_swp = p_add, p_cmp = p_add)
    # Store exit codes of individual mutation functions
    status = 0
    
    # Determine which mutations to perform
    doMutate = rand(4) .< [p_add, p_del, p_swp, p_cmp]
    
    if doMutate[1]
        status += mutateAddEdge!(sGraph)
    end

    if doMutate[2]
        status += mutateDelEdge!(sGraph.graph)
    end

    if doMutate[3]
        status += mutateSwapEdges!(sGraph)
    end

    if doMutate[4]
        status += mutateCompetitors!(sGraph)
    end

    return(status)
end
  
"""
Add an edge to a structure graph
"""
function mutateAddEdge!(sGraph::structureGraph)
    graph = sGraph.graph
    competition = sGraph.competition
    N = nv(graph)
    
    if ne(graph) >= N*(N-1)/2
        # The graph is already complete
        return(1)
    else
        while true
            (a,b) = ceil.(N*rand(2))
            if (a != b) && (add_edge!(graph, a, b))
                # Do not allow self loop
                # Do not allow links between competitors
                if (competition[a] == competition[b])
                    rem_edge!(graph, Int64(a), Int64(b))
                    return(1)
                end
                # Exit if edge sucessfully added
                return(0)
            end
        end
    end
end

"""
Remove an edge to a structure graph
"""
function mutateDelEdge!(graph::LightGraphs.SimpleGraphs.SimpleGraph)
    edgesList = [e for e in edges(graph)]
    edgesIndicesOrder = randperm(length(edgesList))
    for edgeIndex in edgesIndicesOrder
        edgeToRemove = edgesList[edgeIndex]
        rem_edge!(graph, edgeToRemove)
        if is_connected(graph)
            return(0)
        else
            # So structure graph should be kept connected
            # Therefore we put back in the removed edge
            add_edge!(graph, edgeToRemove)
        end
    end
    
    # No edge can be removed without diconnecting the graph
    return(1)
end

"""
Swap edges in a structure graph
"""
function mutateSwapEdges!(sGraph::structureGraph)
    graph = sGraph.graph
    competition = sGraph.competition
    
    edgesList = [e for e in edges(graph)]
    edgesIndicesOrder = randperm(length(edgesList))
    
    for (indexIndex, edgeIndex) = enumerate(edgesIndicesOrder)
        edge1 = edgesList[edgeIndex]
        edge2 = edgesList[edgesIndicesOrder[1+(indexIndex % length(edgesList))]]
        # Ensure that no self link will be created
        if Tuple(edge1)[1] != Tuple(edge2)[2] && Tuple(edge2)[1] != Tuple(edge1)[2]
            # Start by deleting the old edges
            rem_edge!(graph, edge1)
            rem_edge!(graph, edge2)
            # Then add the new ones if not linking competitors
            if competition[Tuple(edge1)[1]] != competition[Tuple(edge2)[2]]
                add_edge!(graph, Tuple(edge1)[1], Tuple(edge2)[2])
            end
            if competition[Tuple(edge2)[2]] != competition[Tuple(edge1)[2]]
                add_edge!(graph, Tuple(edge2)[1], Tuple(edge1)[2])
            end
            if is_connected(graph)
                return(0)
            else
                # So structure graph should be kept connected
                # Therefore we put back in the removed edges
                add_edge!(graph, edge1)
                add_edge!(graph, edge2)
                # NB: extra edges will stay if any
            end
        end
    end
    
    # No edges can be swapped without diconnecting the graph
    return(1)
end

"""
Mutate competing nodes
"""
function mutateCompetitors!(sGraph::structureGraph)
    graph = sGraph.graph
    competition = sGraph.competition
    
    # Select node to change competition class
    nodeComp = rand(1:nv(graph))
    # Select new competition class
    newComp = rand(1:nv(graph))
    for n = neighbors(graph, nodeComp)
        if competition[n] == newComp
            # Changing the competition class would lead to linked competitors
            return(1)
        end
    end
    competition[nodeComp] = newComp
    
    return(0)
end

"""
Cross-over between two structure graphs
"""
function crossOverGraphs!(sGraph1::structureGraph, sGraph2::structureGraph)
    return(1)
end

## Genetic algorithm module

In [ ]:
"""
Compute loss for a given structure
compared to observation
"""
function observedLoss(sGraph::structureGraph,
    details::Bool = false, alpha::Float64 = ALPHA)
    pGraphARID, pGraphBRG = structureToPulldowns(sGraph)
    
    observedEdgesARID = edgeTypesARID[alpha]
    observedEdgesBRG = edgeTypesBRG[alpha]
    
    intersectEdgesARID = intersect(pGraphARID.edges, observedEdgesARID)
    intersectEdgesBRG = intersect(pGraphBRG.edges, observedEdgesBRG)
    unionEdgesARID = union(pGraphARID.edges, observedEdgesARID)
    unionEdgesBRG = union(pGraphBRG.edges, observedEdgesBRG)
    
    if details
        # Return array with Jaccard index
        # length of union and length of  
        return([(length(intersectEdgesARID)+length(intersectEdgesBRG)) / (length(unionEdgesARID)+length(unionEdgesBRG)),
                length(intersectEdgesARID), length(pGraphARID.edges), length(intersectEdgesBRG), length(pGraphBRG.edges)])
    else
        # Return Jaccard index
        return([(length(intersectEdgesARID)+length(intersectEdgesBRG)) / (length(unionEdgesARID)+length(unionEdgesBRG))])
    end
end

"""
Generate in place the new generation of 
structure graphs based on their fitness.
Return the fitness array.
"""
function reproduceGeneration!(pop::Array{structureGraph,1},
    details::Bool = false)
    jaccard = map(x -> observedLoss(x,details), pop)
    fitness = map(x -> x[1], jaccard)
    fitness ./= sum(fitness)
    
    sumFitness = sum(fitness) 
    if sumFitness != 1
        fitness[end] += 1 - sumFitness
    end
    # Ensure the cumulative fitnesses is a probability distribution
    
    offspringPerGraph = rand(Multinomial(length(pop), fitness), 1)
    offspring = Array{structureGraph,1}(length(pop))
    
    offspringToFill = 1 # Which is the next index to be filled?
    for (ipop, noff) = enumerate(offspringPerGraph)
        for ioff = 1:noff
            offspring[offspringToFill] = deepcopy(pop[ipop])
            offspringToFill += 1
        end
    end
    
    # Ensure the best structure graph is kept
    bestGraphIndex = findmax(fitness)[2]
    if offspringPerGraph[bestGraphIndex] == 0
        # No offspring for the best graph
        # So we force one
        offspring[1] = deepcopy(pop[bestGraphIndex])
    end
    
    pop .= offspring
        
    return(jaccard)
end

"""
Generate the new generation of structure networks
"""
function newGeneration!(pop::Array{structureGraph,1},
        details::Bool = false;
        p_add = 0.1, p_del = p_add, p_swp = p_add, p_cmp = p_add, p_crs = p_add/10)
    # Fitness-based reproduction
    fitness = reproduceGeneration!(pop, details)
    
    # Mutate potentially each structure network
    map(x -> mutateStructureGraph!(x;
            p_add = p_add, p_del = p_del, p_swp = p_swp, p_cmp = p_cmp), pop)
    
    # Cross-over
#     if rand() < p_crs
#         sGraph1 = rand(pop)
#         sGraph2 = rand(pop)
#         if sGraph1 != sGraph2
#             crossOverGraphs!(sGraph1, sGraph2)
#         end
#     end
    
    return(fitness)
end

## Run genetic algorithm

In [ ]:
# Run parameters
const N = 100 # Number of graphs [500, 1000]
const L = 10000 # Number 0f iterations [minimum 2000/1000 needed, 5000, 10000,25000]
const P = 0.026 # Probability of mutation [0.01275, 0.026]
# Expect 10% of graphs mutated per generation

In [ ]:
# Max number of edges in a graph
maxEdges = Int64(M*(M-1)/2)

# Initialize population
pop = map(x -> structureGraph(
        Graph(M, rand(1:maxEdges)),
        copy(unitDictString),
        Dict(e => e for e in 1:M)),
    1:N)

# Ensure connectivity
map(connectGraph!, pop)

In [ ]:
#H ow often should we keep track of the system's state?
monitorStep = 40
    
@time begin
quantileFitness = Array{Float16}(Int(ceil(L/monitorStep)), 5)
quantileIntersectARID = Array{Float16}(Int(ceil(L/monitorStep)), 5)
quantileSimulatedEdgesARID = Array{Float16}(Int(ceil(L/monitorStep)), 5)
quantileIntersectBRG = Array{Float16}(Int(ceil(L/monitorStep)), 5)
quantileSimulatedEdgesBRG = Array{Float16}(Int(ceil(L/monitorStep)), 5)
for i in 1:L
    if i % monitorStep == 1
        f = newGeneration!(pop, true, p_add = P)
        currentStep = Int(ceil(i/monitorStep))
        quantileFitness[currentStep,:] = quantile(map(x -> x[1], f))
        quantileIntersectARID[currentStep,:] = quantile(map(x -> x[2], f))
        quantileSimulatedEdgesARID[currentStep,:] = quantile(map(x -> x[3], f))
        quantileIntersectBRG[currentStep,:] = quantile(map(x -> x[4], f))
        quantileSimulatedEdgesBRG[currentStep,:] = quantile(map(x -> x[5], f))
    else
        f = newGeneration!(pop, false, p_add = P)
    end
end
end

In [ ]:
using JLD, HDF5

save("/Users/lvulliard/tests/BAF_Julia/test.jld","pop", pop,
    "fitness", quantileFitness, "intersectARID", quantileIntersectARID, "quantileSimulatedEdgesARID", quantileSimulatedEdgesARID,
    "intersectBRG", quantileIntersectBRG, "quantileSimulatedEdgesBRG", quantileSimulatedEdgesBRG)

## Monitor results

In [ ]:
indexBestGraph

In [ ]:
indexBestGraph = findmax(map(x -> observedLoss(x, true)[1], pop))[2]
bestStructure = pop[indexBestGraph]
bestPulldownARID, bestPulldownBRG = structureToPulldowns(bestStructure)

In [ ]:
traceFitness = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(5)

for i = 1:5
    traceFitness[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name= string("Top ", 25*(i-1), "%"),
        y= quantileFitness[:,i], mode="lines+markers")
end

layoutFitness = Layout(yaxis_title="<b>Jaccard coefficient distribution</b>", xaxis_title = "<b>Generation</b>")

plot(traceFitness, layoutFitness)

In [ ]:
traceIntersectARID = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(6)

for i = 1:5
    traceIntersectARID[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = string("Top ", 25*(i-1), "%"),
        y= quantileIntersectARID[:,i], mode="lines+markers")
end

traceIntersectARID[6] = scatter(
    x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = "Edges in observed pull-down graph",
    y= map(x -> length(edgeTypesARID[0.05]), 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1)), mode="lines")

layoutIntersectARID = Layout(yaxis_title="<b>Pull-down edges intersection size</b>", xaxis_title = "<b>Generation</b>")

plot(traceIntersectARID, layoutIntersectARID)

In [ ]:
traceSimulatedEdgesARID = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(5)

for i = 1:5
    traceSimulatedEdgesARID[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name= string("Top ", 25*(i-1), "%"),
        y= quantileSimulatedEdgesARID[:,i], mode="lines+markers")
end

layoutSimulatedEdgesARID = Layout(yaxis_title="<b>Number of simulated pull-down edges</b>", xaxis_title = "<b>Generation</b>")

plot(traceSimulatedEdgesARID, layoutSimulatedEdgesARID)

In [ ]:
traceIntersectBRG = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(6)

for i = 1:5
    traceIntersectBRG[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = string("Top ", 25*(i-1), "%"),
        y= quantileIntersectBRG[:,i], mode="lines+markers")
end

traceIntersectBRG[6] = scatter(
    x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = "Edges in observed pull-down graph",
    y= map(x -> length(edgeTypesBRG[0.05]), 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1)), mode="lines")

layoutIntersectBRG = Layout(yaxis_title="<b>Pull-down edges intersection size</b>", xaxis_title = "<b>Generation</b>")

plot(traceIntersectBRG, layoutIntersectBRG)

In [ ]:
traceSimulatedEdgesBRG = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(5)

for i = 1:5
    traceSimulatedEdgesBRG[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name= string("Top ", 25*(i-1), "%"),
        y= quantileSimulatedEdgesBRG[:,i], mode="lines+markers")
end

layoutSimulatedEdgesBRG = Layout(yaxis_title="<b>Number of simulated pull-down edges</b>", xaxis_title = "<b>Generation</b>")

plot(traceSimulatedEdgesBRG, layoutSimulatedEdgesBRG)

### Display infered heatmap

In [ ]:
pdSimData = zeros(length(studyARIDpd), length(studyARIDko))

for (edges, edgeType) = bestPulldownARID.edges
    # Which cell should we fill?
    indexKO = findfirst(studyARIDko, unitDictString[edges[1]])
    indexPD = findfirst(studyARIDpd, unitDictString[edges[2]])
    
    # What type / value for the edge?
    t = edgeType == "inhibits" ? -1 : 1
    
    pdSimData[indexPD, indexKO] = t
end

In [ ]:
tracePdHeatmap = heatmap(
    x=studyARIDpd,
    y=studyARIDko, # NB: filter genes outside of BAF complex
    z=pdSimData
)

stylePdHeatmap = Style(global_trace=attr(colorscale=[[0, "rgb(0,0,255)"], [0.5, "rgb(255,255,255)"], [1, "rgb(255,0,0)"]]))
layoutPdHeatmap = Layout(;margin_l = 100, margin_t = 20, yaxis_title="<b>Knocked-out gene</b>", xaxis_title = "<b>BAF subunit</b>")
plot(tracePdHeatmap, layoutPdHeatmap, style=stylePdHeatmap)

In [ ]:
plot(traceArid, layoutArid, style=styleArid)

In [ ]:
pdSimData = zeros(length(studyBRGpd), length(studyBRGko))

for (edges, edgeType) = bestPulldownBRG.edges
    # Which cell should we fill?
    indexKO = findfirst(studyBRGko, unitDictString[edges[1]])
    indexPD = findfirst(studyBRGpd, unitDictString[edges[2]])
    
    # What type / value for the edge?
    t = edgeType == "inhibits" ? -1 : 1
    
    pdSimData[indexPD, indexKO] = t
end

In [ ]:
tracePdHeatmap = heatmap(
    x=studyBRGpd,
    y=studyBRGko, # NB: filter genes outside of BAF complex
    z=pdSimData
)

stylePdHeatmap = Style(global_trace=attr(colorscale=[[0, "rgb(0,0,255)"], [0.5, "rgb(255,255,255)"], [1, "rgb(255,0,0)"]]))
layoutPdHeatmap = Layout(;margin_l = 100, margin_t = 20, yaxis_title="<b>Knocked-out gene</b>", xaxis_title = "<b>BAF subunit</b>")
plot(tracePdHeatmap, layoutPdHeatmap, style=stylePdHeatmap)

In [ ]:
plot(traceBrg, layoutBrg, style=styleBrg)

## Average on whole population

In [ ]:
# Weight by fitness
popWeight = map(x -> observedLoss(x, true)[1], pop)

In [ ]:
averageComp = Dict{Tuple,Float64}((i,j) => 0 for i in 1:nv(pop[1].graph) for j in 1:nv(pop[1].graph) if i > j)
for i in 1:length(pop)
    graph = pop[i].graph
    competition = pop[i].competition
    for nodeA in 2:nv(graph)
        for nodeB in 1:nv(graph)
            if nodeA > nodeB && competition[nodeA] == competition[nodeB]
                averageComp[(nodeA,nodeB)] += popWeight[i]
            end
        end
    end
end
                
# Remove null values
averageComp = Dict(c => v/sum(popWeight) for (c,v) in averageComp if v > 0)

In [ ]:
# Cumulated weights of the graph having each edge
averageEdges = Dict{Tuple,Float64}((i,j) => 0 for i in 1:nv(pop[1].graph) for j in 1:nv(pop[1].graph) if i != j)
for i in 1:length(pop)
    for c = edges(pop[i].graph)
        averageEdges[Tuple(c)] += popWeight[i]
    end
end
                
# Remove null values
averageEdges = Dict(c => v/sum(popWeight) for (c,v) in averageEdges if v != 0)

### Export graph with two weighted edge types

In [ ]:
fileGML = open("ARID_average_structure.gml", "w")
saveLabeledGml(fileGML, bestPulldown.graph, bestPulldown.nodes, averageEdges, averageComp)
close(fileGML)

## Alternative initial conditions

### Competition classes from sequence similarity

In [ ]:
# Max number of edges in a graph
maxEdges = Int64(M*(M-1)/2)

# Litterature competitions
compDictLitt = Dict(unitDict["SMARCA4"] => 1,
    unitDict["SMARCA2"] => 1,
    unitDict["ARID1A"] => 2,
    unitDict["ARID1B"] => 2,
    unitDict["SMARCD1"] => 3,
    unitDict["SMARCD2"] => 3,
    unitDict["SMARCD3"] => 3,
    unitDict["DPF1"] => 4,
    unitDict["DPF2"] => 4,
    unitDict["DPF3"] => 4,
    unitDict["SMARCC1"] => 5,
    unitDict["SMARCC2"] => 5,
    unitDict["SS18"] => 7,
    unitDict["SS18L1"] => 7,
    unitDict["BCL11A"] => 8,
    unitDict["BCL11B"] => 8,
    unitDict["ACTL6A"] => 9,
    unitDict["ACTL6B"] => 9
)

# Initialize population
pop2 = map(x -> structureGraph(
        Graph(M, rand(1:maxEdges)),
        copy(unitDictString),
        copy(compDictLitt)),
    pop2)

# Ensure connectivity
map(randomCompetitionGraph!, pop2)
map(connectGraph!, pop2)

### Litterature-based initial interactions

In [ ]:
# Max number of edges in a graph
maxEdges = Int64(M*(M-1)/2)

# Litterature competitions
compDictLitt = Dict(unitDict["SMARCA4"] => 1,
    unitDict["SMARCA2"] => 1,
    unitDict["ARID1A"] => 2,
    unitDict["ARID1B"] => 2,    
    unitDict["ARID2"] => 2,
    unitDict["SMARCD1"] => 3,
    unitDict["SMARCD2"] => 3,
    unitDict["SMARCD3"] => 3,
    unitDict["PHF10"] => 4,
    unitDict["DPF1"] => 4,
    unitDict["DPF2"] => 4,
    unitDict["DPF3"] => 4,
    unitDict["SMARCC1"] => 5,
    unitDict["SMARCC2"] => 5,
    unitDict["BCL7A"] => 6,
    unitDict["BCL7B"] => 6,
    unitDict["BCL7C"] => 6,
    unitDict["SS18"] => 7,
    unitDict["SS18L1"] => 7,
    unitDict["BCL11A"] => 8,
    unitDict["BCL11B"] => 8,
    unitDict["ACTL6A"] => 9,
    unitDict["ACTL6B"] => 9
)

graphLitt = Graph(M)
add_edge!(graphLitt, unitDict["SMARCC1"], unitDict["SMARCB1"])
add_edge!(graphLitt, unitDict["SMARCE1"], unitDict["SMARCC1"])
add_edge!(graphLitt, unitDict["SMARCE1"], unitDict["SMARCC2"])
add_edge!(graphLitt, unitDict["SMARCA4"], unitDict["ACTB"])
add_edge!(graphLitt, unitDict["SMARCA4"], unitDict["ACTL6A"])
add_edge!(graphLitt, unitDict["SMARCA4"], unitDict["ACTL6B"])
add_edge!(graphLitt, unitDict["SMARCA2"], unitDict["ACTB"])
add_edge!(graphLitt, unitDict["SMARCA2"], unitDict["ACTL6A"])
add_edge!(graphLitt, unitDict["SMARCA2"], unitDict["ACTL6B"])

# Initialize population\
pop2 = map(x -> structureGraph(
        deepcopy(graphLitt),
        copy(unitDictString),
        copy(compDictLitt)),
    pop2)

# Ensure connectivity
map(randomCompetitionGraph!, pop2)
map(connectGraph!, pop2)

pop2

### Similarity matrix

In [ ]:
"""
Results from cluster: (remove line carriages)
simiMatrix = [1.0 0.207006 0.245482 0.128962 0.211506 0.167286 0.160516 0.633574 0.207006 0.251582 0.233227 0.68797 0.123537
    0.235974 0.222581 0.241908 0.232468 0.231383 0.188136 0.140107 0.206501 0.191045 0.238095 0.192079 0.19788 0.24957 0.155979
    0.170489 0.244698; 0.208599 1.0 0.209192 0.125541 0.600462 0.133497 0.134977 0.199422 0.18465 0.204545 0.195423 0.200297
    0.125272 0.203267 0.207581 0.2 0.192612 0.184507 0.179688 0.141088 0.184946 0.197674 0.212544 0.166667 0.19604 0.188645
    0.141774 0.161832 0.184919; 0.238872 0.209192 1.0 0.126785 0.209756 0.157509 0.154249 0.239832 0.214592 0.238547 0.239234
    0.243205 0.125271 0.247947 0.278523 0.224456 0.239596 0.219346 0.199321 0.143697 0.201479 0.205231 0.224843 0.194175 0.193825
    0.241042 0.156682 0.178899 0.273032; 0.128962 0.125541 0.126352 1.0 0.118004 0.215679 0.214984 0.139404 0.17138 0.108658
    0.112847 0.135439 0.510444 0.111979 0.111785 0.105355 0.133047 0.137484 0.186358 0.226488 0.0732984 0.18314 0.107281 0.0675087
    0.0873533 0.105401 0.213578 0.18419 0.104737; 0.211506 0.599078 0.209756 0.118004 1.0 0.129032 0.125734 0.21519 0.193296 0.233512
    0.206522 0.210031 0.116855 0.205405 0.226843 0.211429 0.19837 0.205382 0.174978 0.137669 0.216401 0.177335 0.228417 0.197727 
    0.191983 0.20566 0.136817 0.153485 0.188406; 0.167286 0.133578 0.157605 0.21573 0.129273 1.0 0.726394 0.186847 0.194958 0.145679 
    0.146221 0.169765 0.222878 0.155376 0.141971 0.132262 0.177177 0.17491 0.214169 0.21491 0.0969356 0.204307 0.139765 0.09 0.108628 
    0.138547 0.226012 0.208075 0.14532; 0.160516 0.134977 0.154249 0.214415 0.125734 0.726238 1.0 0.181555 0.195931 0.13783 0.142019 
    0.165015 0.234224 0.151622 0.132042 0.127422 0.181612 0.169714 0.215104 0.216478 0.094362 0.197906 0.140598 0.0877817 0.110913 
    0.130715 0.22488 0.208394 0.144191; 0.633574 0.199422 0.238095 0.139404 0.21643 0.186847 0.181555 1.0 0.225572 0.24805 0.233129 
    0.653571 0.141746 0.22884 0.2118 0.242928 0.230769 0.219451 0.198826 0.150026 0.189807 0.216056 0.246554 0.181004 0.194847 
    0.223089 0.156751 0.179026 0.232666; 0.206383 0.184855 0.214592 0.171236 0.193296 0.194507 0.195513 0.225572 1.0 0.206593 
    0.19214 0.201903 0.171429 0.201105 0.198661 0.174302 0.221574 0.228426 0.223388 0.195455 0.153121 0.608879 0.213816 0.146714 
    0.167637 0.184902 0.204833 0.226415 0.210699; 0.251981 0.208042 0.238547 0.108658 0.236655 0.145679 0.13783 0.24805 0.205689 
    1.0 0.227671 0.233487 0.111642 0.231834 0.235081 0.37744 0.228883 0.21458 0.175454 0.139069 0.224576 0.195344 0.842593 0.198276 
    0.221557 0.239209 0.144944 0.164877 0.24237; 0.227564 0.195423 0.240711 0.112847 0.206522 0.145511 0.142019 0.233129 0.19214 
    0.227671 1.0 0.215267 0.112413 0.246914 0.411647 0.242481 0.206851 0.208333 0.177193 0.12224 0.192225 0.197505 0.24237 0.183036 
    0.216667 0.226345 0.13785 0.164988 0.482688; 0.68797 0.200297 0.242165 0.135439 0.208791 0.170476 0.164822 0.653571 0.203158 
    0.233487 0.215267 1.0 0.129381 0.235679 0.224359 0.242833 0.229529 0.223702 0.186085 0.144149 0.194495 0.20297 0.237097 0.186567 
    0.19637 0.232595 0.15634 0.174347 0.22807; 0.123537 0.125272 0.125271 0.510824 0.117391 0.223373 0.234311 0.141746 0.171429 
    0.11087 0.112413 0.129381 1.0 0.111836 0.106817 0.104257 0.141026 0.139776 0.174868 0.22823 0.0747704 0.193737 0.10913 0.0720524 
    0.0820602 0.106087 0.201242 0.191216 0.108988; 0.235489 0.203267 0.247947 0.111979 0.205405 0.156328 0.151622 0.231496 0.201105 
    0.231834 0.246914 0.234811 0.111836 1.0 0.217626 0.229478 0.23594 0.222543 0.180299 0.121846 0.210177 0.180513 0.230241 0.207965 
    0.202062 0.235622 0.146113 0.168232 0.20654; 0.223301 0.207581 0.278523 0.111304 0.226843 0.141708 0.132042 0.2118 0.197998 
    0.233273 0.411647 0.224359 0.105995 0.217626 1.0 0.243494 0.211022 0.209169 0.16509 0.124798 0.213004 0.195173 0.234432 0.210884 
    0.223849 0.229358 0.141279 0.160187 0.545455; 0.241908 0.200737 0.223154 0.105263 0.209924 0.132262 0.127422 0.242928 0.175223 
    0.383117 0.242481 0.246667 0.104212 0.229478 0.243494 1.0 0.197479 0.21988 0.159649 0.1195 0.211401 0.182773 0.380435 0.194712 
    0.232104 0.241843 0.128938 0.155294 0.238447; 0.233463 0.192612 0.239899 0.133047 0.19837 0.176188 0.181612 0.230769 0.222006 
    0.227086 0.206851 0.229529 0.140899 0.23594 0.213026 0.197479 1.0 0.3687 0.222494 0.173706 0.174298 0.218808 0.233379 0.168405 
    0.189189 0.204545 0.188851 0.212166 0.208995; 0.231383 0.184922 0.219346 0.137424 0.204255 0.175407 0.169714 0.219451 0.228195 
    0.21458 0.208333 0.221929 0.139776 0.222543 0.210678 0.21988 0.368212 1.0 0.217464 0.165869 0.203252 0.224784 0.221127 0.171799 
    0.218553 0.211731 0.17531 0.202256 0.212079; 0.189143 0.179688 0.199321 0.186434 0.176471 0.214169 0.214062 0.198826 0.223221 
    0.178664 0.177193 0.187081 0.174694 0.180299 0.16595 0.159649 0.222675 0.217464 1.0 0.21334 0.136568 0.227338 0.184324 0.127469 
    0.147294 0.175439 0.238547 0.581749 0.163404; 0.140107 0.141088 0.143697 0.223938 0.137127 0.213899 0.216741 0.150026 0.196465 
    0.137372 0.12224 0.144149 0.22815 0.121912 0.124865 0.1195 0.173798 0.165157 0.213621 1.0 0.0924918 0.203008 0.140541 0.0898204 
    0.106775 0.12581 0.21841 0.217719 0.118123; 0.206501 0.184946 0.201479 0.0732984 0.216401 0.0969356 0.094362 0.189807 0.153121 
    0.224359 0.193966 0.194495 0.0747051 0.210177 0.212054 0.211401 0.175258 0.203252 0.136445 0.0924918 1.0 0.150985 0.228448 0.324528 
    0.319527 0.213152 0.100413 0.122709 0.185897; 0.191426 0.196842 0.206269 0.182947 0.178082 0.205638 0.197906 0.214286 0.608879 
    0.195939 0.198339 0.199802 0.192997 0.179567 0.195173 0.182105 0.218808 0.225 0.226062 0.202899 0.150985 1.0 0.196099 0.13956 
    0.170467 0.184293 0.211777 0.228224 0.194617; 0.236542 0.213542 0.224843 0.107281 0.228417 0.139765 0.140598 0.246177 0.213582
    0.842593 0.240072 0.23435 0.109952 0.228916 0.236264 0.382932 0.236092 0.221127 0.184324 0.140541 0.228448 0.196099 1.0 0.194748 
    0.216599 0.245009 0.143521 0.16834 0.235612; 0.192913 0.166667 0.194175 0.0675087 0.197727 0.0898876 0.0877817 0.181004 0.146714 
    0.197849 0.183036 0.186567 0.0720524 0.206667 0.209459 0.194712 0.168405 0.171799 0.127469 0.0898204 0.32342 0.139407 0.194748 
    1.0 0.292169 0.21729 0.0920354 0.114035 0.184211; 0.19788 0.201195 0.190476 0.0873533 0.191983 0.108628 0.110913 0.194847 0.167442 
    0.221557 0.216667 0.19637 0.0820244 0.202062 0.224066 0.227766 0.18892 0.21821 0.147919 0.106775 0.319527 0.170467 0.215886 0.292169 
    1.0 0.220807 0.11385 0.138013 0.230924; 0.249141 0.188645 0.241042 0.10579 0.204934 0.139692 0.130486 0.222741 0.184498 0.239209 
    0.226345 0.232227 0.106337 0.235622 0.229358 0.241379 0.203481 0.211429 0.175439 0.12581 0.213152 0.184953 0.245009 0.218824 
    0.220807 1.0 0.138533 0.168478 0.235612; 0.155491 0.141774 0.156196 0.2135 0.136817 0.224943 0.225989 0.15729 0.205721 0.145114 
    0.137347 0.155852 0.201681 0.146199 0.141279 0.128938 0.188958 0.176172 0.239244 0.218593 0.0993495 0.211892 0.143521 0.0921986 
    0.11385 0.137529 1.0 0.238956 0.139019; 0.170489 0.161956 0.178271 0.184047 0.153605 0.20788 0.208496 0.179082 0.22549 0.164877 
    0.165244 0.173047 0.191216 0.168363 0.160187 0.157109 0.213066 0.202112 0.581749 0.217514 0.121212 0.228591 0.16834 0.114035 
    0.137549 0.167963 0.239578 1.0 0.160714; 0.24513 0.184919 0.272575 0.104783 0.184116 0.14532 0.14319 0.233846 0.210699 0.244123
    0.482688 0.22807 0.108941 0.213058 0.545455 0.233151 0.208995 0.211781 0.163404 0.117996 0.185897 0.194617 0.243292 0.184211 
    0.232323 0.234234 0.139019 0.159566 1.0]
unitList = String["SMARCD3", "SS18", "PHF10", "ARID1B", "SS18L1", "SMARCA2", "SMARCA4", "SMARCD2", "BCL11A", "ACTL6A", "DPF3", 
"SMARCD1", "ARID1A", "SMARCE1", "DPF2", "ACTB", "BRD7", "BRD9", "SMARCC1", "ARID2", "BCL7A", "BCL11B", "ACTL6B", "BCL7B", 
"BCL7C", "SMARCB1", "PBRM1", "SMARCC2", "DPF1"]
"""

In [ ]:
distMatrix = 1 - simiMatrix
for x = 1:size(distMatrix)[1]
    for y = 1:size(distMatrix)[2]
        if x > y
            if distMatrix[x,y] - distMatrix[y,x] > 0.007
                println(x, '-', y) # Not happening, differences are small
            end
            distMatrix[x,y] = distMatrix[y,x]
        end
    end
end

In [ ]:
distMatrix = 1 - simiMatrix
for x = 1:size(distMatrix)[1]
    for y = 1:size(distMatrix)[2]
        if x > y
            if distMatrix[x,y] - distMatrix[y,x] > 0.007
                println(x, '-', y) # Not happening, differences are small
            end
            distMatrix[x,y] = distMatrix[y,x]
        end
    end
end

In [ ]:
fileSimilarity = open("similarityMatrix.csv", "w")
CSV.write(fileSimilarity, DataFrame(simiMatrix, Symbol.(unitList)))
close(fileSimilarity)

In [ ]:
traceSimilarity = heatmap(
    x=unitList[orderDistMatrix],
    y=unitList[orderDistMatrix],
    z=simiMatrix[orderDistMatrix,orderDistMatrix]
)

styleSimilarity = Style(global_trace=attr(colorscale=[[0, "rgb(255,255,255)"], [1, "rgb(0,85,100)"]]))
layoutSimilarity = Layout(;font_family="arial", font_size=10, margin_l = 100, margin_t = 20, margin_b = 60, xaxis_tickangle = -40, 
    xaxis_title = "<b>BAF/PBAF subunit</b>")
p = plot(traceSimilarity, layoutSimilarity, style=styleSimilarity)

In [ ]:
savefig(p, "similarityMatrix.svg")

## Average across all runs

In [ ]:
using JLD, HDF5

# Load all simulations
pop_runs = Dict{String, Dict{Float64, Array{Dict}}}()
folder = "/Users/lvulliard/tests/BAF_Julia/Archive/OutBAFPBAF/"
all_files = [i for i in readdir(folder) if contains(i, ".jld")]
for ini = ["_simi_", "_rand_", "_litt_"]
    files_ini = [i for i in all_files if contains(i, ini)]
    pop_runs[ini] = Dict{Float64, Array{Dict}}()
    for alpha = [0.1, 0.05, 0.01, 0.005]
        alpha_motif = "_"*replace(string(alpha), ".", "")*"_"
        files_alpha = [i for i in files_ini if contains(i, alpha_motif)]
        pop_runs[ini][alpha] = Array{Dict}(25)
        for (index, file) = enumerate(files_alpha)
            pop_runs[ini][alpha][index] = load(folder*file)
        end
    end
end

In [ ]:
# Look at distribution of final fitnesses for each condition
[(ini, alpha, quantile([popl["fitness"][end] for popl in arrayAlpha])) for (ini, dictIni) in pop_runs for (alpha, arrayAlpha) in dictIni]

### Observe best simulation

In [ ]:
bestFitness = .0
pop = 0
indexBestGraph = 0
indexBestPop = 0

for i = 1:25
    fitness, bestIndex = findmax(map(x -> observedLoss(x, false, 0.005)[1], pop_runs["_rand_"][0.005][i]["pop"]))
    if fitness > bestFitness
        bestFitness = fitness
        pop = pop_runs["_rand_"][0.005][i]["pop"]
        indexBestGraph = bestIndex
        indexBestPop = i
    end
end

In [ ]:
bestStructure = pop[indexBestGraph]
bestPulldownARID, bestPulldownBRG = structureToPulldowns(bestStructure)

In [ ]:
quantileFitness = pop_runs["_rand_"][0.005][indexBestPop]["fitness"]
quantileIntersectBRG = pop_runs["_rand_"][0.005][indexBestPop]["intersectBRG"]
quantileIntersectARID = pop_runs["_rand_"][0.005][indexBestPop]["intersectARID"]
quantileSimulatedEdgesARID = pop_runs["_rand_"][0.005][indexBestPop]["quantileSimulatedEdgesARID"]
quantileSimulatedEdgesBRG = pop_runs["_rand_"][0.005][indexBestPop]["quantileSimulatedEdgesBRG"]
L = 10000
monitorStep = 50

In [ ]:
traceFitness = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(5)

for i = 1:5
    traceFitness[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name= string("Top ", 25*(i-1), "%"),
        y= quantileFitness[:,i], mode="lines+markers")
end

layoutFitness = Layout(yaxis_title="<b>Jaccard coefficient distribution</b>", xaxis_title = "<b>Generation</b>")

plot(traceFitness, layoutFitness)

In [ ]:
traceIntersectARID = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(6)

for i = 1:5
    traceIntersectARID[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = string("Top ", 25*(i-1), "%"),
        y= quantileIntersectARID[:,i], mode="lines+markers")
end

traceIntersectARID[6] = scatter(
    x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = "Edges in observed pull-down graph",
    y= map(x -> length(edgeTypesARID[0.05]), 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1)), mode="lines")

layoutIntersectARID = Layout(yaxis_title="<b>Pull-down edges intersection size</b>", xaxis_title = "<b>Generation</b>")

plot(traceIntersectARID, layoutIntersectARID)

In [ ]:
traceSimulatedEdgesARID = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(5)

for i = 1:5
    traceSimulatedEdgesARID[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name= string("Top ", 25*(i-1), "%"),
        y= quantileSimulatedEdgesARID[:,i], mode="lines+markers")
end

layoutSimulatedEdgesARID = Layout(yaxis_title="<b>Number of simulated pull-down edges</b>", xaxis_title = "<b>Generation</b>")

plot(traceSimulatedEdgesARID, layoutSimulatedEdgesARID)

In [ ]:
traceIntersectBRG = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(6)

for i = 1:5
    traceIntersectBRG[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = string("Top ", 25*(i-1), "%"),
        y= quantileIntersectBRG[:,i], mode="lines+markers")
end

traceIntersectBRG[6] = scatter(
    x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name = "Edges in observed pull-down graph",
    y= map(x -> length(edgeTypesBRG[0.05]), 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1)), mode="lines")

layoutIntersectBRG = Layout(yaxis_title="<b>Pull-down edges intersection size</b>", xaxis_title = "<b>Generation</b>")

plot(traceIntersectBRG, layoutIntersectBRG)

In [ ]:
traceSimulatedEdgesBRG = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(5)

for i = 1:5
    traceSimulatedEdgesBRG[i] = scatter(
        x= 1+monitorStep*((1:Int(ceil(L/monitorStep)))-1), name= string("Top ", 25*(i-1), "%"),
        y= quantileSimulatedEdgesBRG[:,i], mode="lines+markers")
end

layoutSimulatedEdgesBRG = Layout(yaxis_title="<b>Number of simulated pull-down edges</b>", xaxis_title = "<b>Generation</b>")

plot(traceSimulatedEdgesBRG, layoutSimulatedEdgesBRG)

### Compute and export average

In [ ]:
# Weight by fitness
# Store weights for all initial conditions
popWeights = Dict{String, Dict{Float64,Array{Array{Float64}}}}()
sumWeights = .0

for (ini, iniDict) = pop_runs
    # Store weights for all alpha thresholds
    popWeights[ini] = Dict{Float64,Array{Array{Float64}}}()
    println(ini)
    for (alpha, alphaArray) = iniDict
        # Store weights for each simulation
        popWeights[ini][alpha] = Array{Array{Float64}}(25)
        println(alpha)
        for (indexSim, sim) = enumerate(alphaArray)
            popWeights[ini][alpha][indexSim] = map(x -> observedLoss(x, false, alpha)[1], sim["pop"])
            sumWeights += popWeights[ini][alpha][indexSim]
        end
    end
end

sumWeights = sum(sumWeights)

In [ ]:
averageComp = Dict{Tuple,Float64}((i,j) => 0 for i in 1:M for j in 1:M if i > j)
                
for (ini, iniDict) = pop_runs
    println(ini)
    for (alpha, alphaArray) = iniDict
        println(alpha)
        for (indexSim, sim) = enumerate(alphaArray)
            for (indexPop, pop) in enumerate(sim["pop"])
                graph = pop.graph
                competition = pop.competition
                for nodeA in 2:M
                    for nodeB in 1:M
                        if nodeA > nodeB && competition[nodeA] == competition[nodeB]
                            averageComp[(nodeA,nodeB)] += popWeights[ini][alpha][indexSim][indexPop]
                        end
                    end
                end
            end
        end
    end
end
                
# Remove null values
averageComp = Dict(c => v/sumWeights for (c,v) in averageComp if v > 0)

In [ ]:
# Cumulated weights of the graph having each edge
averageEdges = Dict{Tuple,Float64}((i,j) => 0 for i in 1:M for j in 1:M if i != j)

for (ini, iniDict) = pop_runs
    println(ini)
    for (alpha, alphaArray) = iniDict
        println(alpha)
        for (indexSim, sim) = enumerate(alphaArray)
            for (indexPop, pop) in enumerate(sim["pop"])
                for c = edges(pop.graph)
                    # Edge's nodes are always sorted
                    averageEdges[Tuple(c)] += popWeights[ini][alpha][indexSim][indexPop]
                end
            end
        end
    end
end
                
# Remove null values
averageEdges = Dict(c => v/sumWeights for (c,v) in averageEdges if v != 0)

In [ ]:
fileGML = open("BAFPBAF_average_structure.gml", "w")
saveLabeledGml(fileGML, bestStructure.graph, bestStructure.nodes, averageEdges, averageComp)
close(fileGML)

In [ ]:
averageMat = Array{Float64}(M,M)
averageMat .= 0

# Alphabetical order of units
alphaOrderUnits = Dict(v => i for (i,v) in enumerate(sort(unitList)))
aoUnit = function(x::Int64)
    return(alphaOrderUnits[bestStructure.nodes[x]])
end

# Fill competition under the diagonal of the matrix
# i.e. x < y
for (t,v) = averageComp
    x,y = sort(aoUnit.(collect(t)))
    averageMat[x,y] = v
end

# Fill connections above the diagonal of the matrix
# i.e. x > y
for (t,v) = averageEdges
    y,x = sort(aoUnit.(collect(t)))
    averageMat[x,y] = -v
end

traceAverage = heatmap(
    x=sort(unitList),
    y=sort(unitList),
    z=averageMat)

styleAverage = Style(global_trace=attr(colorscale=[[0, "rgb(0,140,160)"],
            [minimum(averageMat)/(minimum(averageMat)-maximum(averageMat)), "rgb(255,255,255)"], [1, "rgb(210,50,60)"]]))
layoutAverage = Layout(;margin_l = 90, margin_t = 5, margin_b = 80, yaxis_title="", xaxis_tickangle = -45,
    xaxis_title = "<b>Interaction</b>", yaxis_title = "<b>Competition</b>", font_family="arial", font_size=10)

p = plot(traceAverage, layoutAverage, style=styleAverage)

In [ ]:
savefig(p, "subD.svg")

In [ ]:
# Get minimal and maximal fitnesses
minWeight, maxWeight = (1,0)

for (ini, iniDict) = pop_runs
    # Store weights for all alpha thresholds
    println(ini)
    for (alpha, alphaArray) = iniDict
        # Store weights for each simulation
        println(alpha)
        for (indexSim, sim) = enumerate(alphaArray)
            mini = minimum(popWeights[ini][alpha][indexSim])
            Maxi = maximum(popWeights[ini][alpha][indexSim])
            minWeight = (mini < minWeight) ? mini : minWeight
            maxWeight = (Maxi > maxWeight) ? Maxi : maxWeight
        end
    end
end

In [ ]:
versioninfo()

In [ ]:
Pkg.status()

In [ ]:
NBSHUFFLE = 1000000

## Analyze runs

In [ ]:
using JLD
simDir = "/Volumes/lvulliard/Documents/BAF-structure/"
allSimFiles = [i for i in readdir(simDir) if contains(i, "run_shuffle")]
simSummaries = Array{Dict}(length(allSimFiles))
for (i, file) in enumerate(allSimFiles)
    simSummaries[i] = load(simDir*file)
end

In [ ]:
nbRuns = length(simSummaries)
N = 500
weightSim = 1/(nbRuns*N)

In [ ]:
averageComp = Dict{Tuple,Float64}((i,j) => 0 for i in 1:M for j in 1:M if i > j)

for runs in simSummaries
    for sim in runs["pop"]
        graph = sim.graph
        competition = sim.competition
        for nodeA in 2:M
            for nodeB in 1:M
                if nodeA > nodeB && competition[nodeA] == competition[nodeB]
                    averageComp[(nodeA,nodeB)] += weightSim
                end
            end
        end
    end
end
                
# Remove null values
averageComp = Dict(c => v for (c,v) in averageComp if v > 0)

In [ ]:
sortedEdgeComp = sort([(v,k) for (k,v) in averageComp])

In [ ]:
for (k,v) in reverse(sortedEdgeComp)
    println([unitDictString[u] for u in v])
end

In [ ]:
averageEdges = Dict{Tuple,Float64}((i,j) => 0 for i in 1:M for j in 1:M if i < j)

for runs in simSummaries
    for sim in runs["pop"]
        for c = edges(sim.graph)
            # Edge's nodes are always sorted
            averageEdges[Tuple(c)] += weightSim
        end
    end
end
                
# Remove null values
averageEdges = Dict(c => v for (c,v) in averageEdges if v > 0)

In [ ]:
sortedEdges = sort([(v,k) for (k,v) in averageEdges])

In [ ]:
for (k,v) in reverse(sortedEdges)
    println([unitDictString[u] for u in v])
end

In [ ]:
traceHistEdges = histogram(x=map(x -> x[1], sortedEdges), opacity=0.75, name="Shuf. interaction rate",
    histnorm="probability", marker_color = "rgb(0,140,160)", nbinsx = 20)
traceHistComp = histogram(x=map(x -> x[1], sortedEdgeComp), opacity=0.75, name="Shuf. competition rate",
    histnorm="probability", marker_color = "rgb(210,50,60)", nbinsx = 20)
data = [traceHistEdges, traceHistComp]
layout = Layout(barmode="overlay")
plot(data, layout)

### Compare to experimental data

In [ ]:
using JLD, HDF5

# Load all simulations
pop_runs = Dict{String, Dict{Float64, Array{Dict}}}()
folder = "/Users/lvulliard/OneShotProject/BAF_Julia/Archive/OutBAFPBAF/"
all_files = [i for i in readdir(folder) if contains(i, ".jld")]
for ini = ["_simi_", "_rand_", "_litt_"]
    files_ini = [i for i in all_files if contains(i, ini)]
    pop_runs[ini] = Dict{Float64, Array{Dict}}()
    for alpha = [0.1, 0.05, 0.01, 0.005]
        alpha_motif = "_"*replace(string(alpha), ".", "")*"_"
        files_alpha = [i for i in files_ini if contains(i, alpha_motif)]
        pop_runs[ini][alpha] = Array{Dict}(25)
        for (index, file) = enumerate(files_alpha)
            pop_runs[ini][alpha][index] = load(folder*file)
        end
    end
end

In [ ]:
averageComp = Dict{Tuple,Float64}((i,j) => 0 for i in 1:M for j in 1:M if i > j)
                
for (ini, iniDict) = pop_runs
    println(ini)
    for (alpha, alphaArray) = iniDict
        println(alpha)
        for (indexSim, sim) = enumerate(alphaArray)
            for (indexPop, pop) in enumerate(sim["pop"])
                graph = pop.graph
                competition = pop.competition
                for nodeA in 2:M
                    for nodeB in 1:M
                        if nodeA > nodeB && competition[nodeA] == competition[nodeB]
                            averageComp[(nodeA,nodeB)] += 1
                        end
                    end
                end
            end
        end
    end
end
                
sumWeights = (N*3*4*25)
                
# Remove null values
averageComp = Dict(c => v/sumWeights for (c,v) in averageComp if v > 0)

In [ ]:
# Cumulated weights of the graph having each edge
averageEdges = Dict{Tuple,Float64}((i,j) => 0 for i in 1:M for j in 1:M if i != j)

for (ini, iniDict) = pop_runs
    println(ini)
    for (alpha, alphaArray) = iniDict
        println(alpha)
        for (indexSim, sim) = enumerate(alphaArray)
            for (indexPop, pop) in enumerate(sim["pop"])
                for c = edges(pop.graph)
                    # Edge's nodes are always sorted
                    averageEdges[Tuple(c)] += 1
                end
            end
        end
    end
end
                
# Remove null values
averageEdges = Dict(c => v/sumWeights for (c,v) in averageEdges if v != 0)

In [ ]:
traceHistEdgesExp = histogram(x=sort([v for (k,v) in averageEdges]), opacity=0.75, name="Exp. interaction rate", 
    histnorm="probability", marker_color = "rgb(70,190,230)", nbinsx = 20)
traceHistCompExp = histogram(x=sort([v for (k,v) in averageComp]), opacity=0.75, name="Exp. competition rate", 
    histnorm="probability", marker_color = "rgb(255,120,130)", nbinsx = 20)
data = [traceHistEdges, traceHistComp, traceHistEdgesExp, traceHistCompExp]
# layout = Layout(barmode="stack", xaxis_title = "Rate", yaxis_title = "Frequency ", font_family="arial", 
#     xaxis_range = [0.11,1], yaxis_range = [0,0.01])
layout = Layout(barmode="stack", xaxis_title = "Rate", yaxis_type = "log", 
    yaxis_title = "Frequency ", font_family="arial")#, bargroupgap=0.5)
histoShuffle = plot(data, layout)

In [ ]:
using Rsvg
savefig(histoShuffle, "histoDistribFreq.svg")

In [ ]:
Pkg.add("Rsvg")

In [ ]:
maximum(sortedEdgeComp)

In [ ]:
maximum(sortedEdges)

In [ ]:
begFit = 0
endFit = 0      
                
for (ini, iniDict) = pop_runs
    for (alpha, alphaArray) = iniDict
        for (indexSim, sim) = enumerate(alphaArray)
            fitness = sim["fitness"]
            begFit += fitness[1,3]
            endFit += fitness[end,3]            
        end
    end
end
                
nbSim = (3*4*25)
          
begFit /= nbSim
endFit /= nbSim

In [ ]:
begFitShuffle = 0
endFitShuffle = 0   

for runs in simSummaries
    fitness = runs["fitness"]
    begFitShuffle += fitness[1,3]
    endFitShuffle += fitness[end,3] 
end

nbSim = (3*4*15)

begFitShuffle /= nbSim
endFitShuffle /= nbSim

In [ ]:
println(begFitShuffle)
println(endFitShuffle)
println(begFit)
println(endFit)

## Compare synthetic lethalities to inferred subunits connectivity

In [ ]:
global const ALPHA_SYNTH = 0.001

In [ ]:
syntheticDF = CSV.read("20171120_inclOutliers_inclHits_3.txt"; delim='\t')
names(syntheticDF)

In [ ]:
# Keep only rows with viability values
syntheticDF = syntheticDF[.!ismissing.(syntheticDF[:deltaPOC_noOutliers]),:]
# Filter on p-values and keep only columns needed
syntheticDFfiltered = syntheticDF[syntheticDF[:pValue_noOutliers] .<
    ALPHA_SYNTH,:]
#    ALPHA_SYNTH,[:CellLine, :GeneSymbol, :POC_noOutliers, :deltaPOC_noOutliers]]
# Remove A549 cell lines
syntheticDFfiltered = syntheticDFfiltered[.!contains.(syntheticDFfiltered[:CellLine], "A549"),:]
categorical!(syntheticDFfiltered, :CellLine)
# Recode the cell lines to the KOed gene
recode!(syntheticDFfiltered[:CellLine], "WT parental"=>"WT", "SMARCA4 4"=>"SMARCA4", "SMARCA4 6"=>"SMARCA4",
    "ARID1A 3"=>"ARID1A",  "ARID1A 10"=>"ARID1A", "ACTIN"=>"ACTB")
# Only keep levels in use
droplevels!(syntheticDFfiltered[:CellLine])

In [ ]:
# KD genes are already formatted as needed
allKO = split.(syntheticDFfiltered[:,:GeneSymbol], '_')
levels(CategoricalArray(vcat(allKO...)))

In [ ]:
# Combine KOed and KDed genes in one field
syntheticDFfiltered[:Genes] = map(x -> unique(Array{String}([x[:CellLine], split(x[:GeneSymbol], '_')...])), eachrow(syntheticDFfiltered))

In [ ]:
# Initialize all weights to 0
syntheticDFfiltered[:Weight] = 0
syntheticDFfiltered[:Weight] = Float64.(syntheticDFfiltered[:Weight])

# For each sample, compute the sum of the disrupted interactions
for (sampleIndex, sample) in enumerate(syntheticDFfiltered[:Genes])
    sampleIndices = [unitDict[gene] for gene in sample]
    for gene in sample
        geneIndex = unitDict[gene]
        for u in 1:M
            # Keys in averageEdges are ordered
            tupleKey = u < geneIndex ? (u,geneIndex) : (geneIndex,u)
            if tupleKey in keys(averageEdges) && !(u in sampleIndices)
                syntheticDFfiltered[sampleIndex,:Weight] += averageEdges[tupleKey]
            end
        end
    end
end

In [ ]:
traceSynthetic = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(1)

traceSynthetic[1] = scatter(
    x=syntheticDFfiltered[:Weight],
    y=syntheticDFfiltered[:deltaPOC_noOutliers],
    marker_color = length.(syntheticDFfiltered[:Genes]),
    mode="markers"
)
layoutSynthetic = Layout(xaxis_title="<b>Sum of disrupted weights</b>", yaxis_title = "<b>Delta(POC)</b>")

plot(traceSynthetic, layoutSynthetic)

In [ ]:
traceSynthetic = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(1)

traceSynthetic[1] = scatter(
    x=syntheticDFfiltered[:Weight],
    y=syntheticDFfiltered[:POC_noOutliers],
    marker_color = length.(syntheticDFfiltered[:Genes]),
    mode="markers"
)
layoutSynthetic = Layout(xaxis_title="<b>Sum of disrupted weights</b>", yaxis_title = "<b>POC</b>")

plot(traceSynthetic, layoutSynthetic)

In [ ]:
println(cor(syntheticDFfiltered[:Weight],syntheticDFfiltered[:deltaPOC_noOutliers]))
println(cor(syntheticDFfiltered[:Weight],syntheticDFfiltered[:POC_noOutliers]))
println(cor(syntheticDFfiltered[:deltaPOC_noOutliers],syntheticDFfiltered[:POC_noOutliers]))

In [ ]:
# Initialize all weights to 0
syntheticDFfiltered[:RemWeight] = 0
syntheticDFfiltered[:RemWeight] = Float64.(syntheticDFfiltered[:RemWeight])

# For each sample, compute the sum of the disrupted interactions
for (sampleIndex, sample) in enumerate(syntheticDFfiltered[:Genes])
    sampleIndices = [unitDict[gene] for gene in sample]
    for u in 1:M if !(u in sampleIndices)
        for v in (u+1):M if !(v in sampleIndices)
            tupleKey = (u,v)
            if tupleKey in keys(averageEdges)
                syntheticDFfiltered[sampleIndex,:RemWeight] += averageEdges[tupleKey]
            end
        end end
    end end
end

In [ ]:
traceSynthetic = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(1)

traceSynthetic[1] = scatter(
    x=syntheticDFfiltered[:RemWeight],
    y=syntheticDFfiltered[:deltaPOC_noOutliers],
    marker_color = length.(syntheticDFfiltered[:Genes]),
    mode="markers"
)
layoutSynthetic = Layout(xaxis_title="<b>Sum of disrupted weights</b>", yaxis_title = "<b>Delta(POC)</b>")

plot(traceSynthetic, layoutSynthetic)

In [ ]:
traceSynthetic = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(1)

traceSynthetic[1] = scatter(
    x=syntheticDFfiltered[:RemWeight],
    y=syntheticDFfiltered[:POC_noOutliers],
    marker_color = length.(syntheticDFfiltered[:Genes]),
    mode="markers"
)
layoutSynthetic = Layout(xaxis_title="<b>Remaining weights</b>", yaxis_title = "<b>POC</b>")

plot(traceSynthetic, layoutSynthetic)

In [ ]:
println(cor(syntheticDFfiltered[:RemWeight],syntheticDFfiltered[:deltaPOC_noOutliers]))
println(cor(syntheticDFfiltered[:RemWeight],syntheticDFfiltered[:POC_noOutliers]))
println(cor(syntheticDFfiltered[:RemWeight],syntheticDFfiltered[:Weight]))

In [ ]:
syntheticDFfiltered

In [ ]:
# Combine KOed and KDed genes in one field
syntheticDFfiltered[:GenesKD] = map(x -> unique(Array{String}([split(x[:GeneSymbol], '_')...])), eachrow(syntheticDFfiltered))

In [ ]:
# Initialize all weights to 0
syntheticDFfiltered[:KdWeight] = 0
syntheticDFfiltered[:KdWeight] = Float64.(syntheticDFfiltered[:KdWeight])

# For each sample, compute the sum of the disrupted interactions
for (sampleIndex, sample) in enumerate(syntheticDFfiltered[:GenesKD])
    sampleIndices = [unitDict[gene] for gene in sample]
    for gene in sample
        geneIndex = unitDict[gene]
        for u in 1:M
            # Keys in averageEdges are ordered
            tupleKey = u < geneIndex ? (u,geneIndex) : (geneIndex,u)
            if tupleKey in keys(averageEdges) && !(u in sampleIndices)
                syntheticDFfiltered[sampleIndex,:KdWeight] += averageEdges[tupleKey]
            end
        end
    end
end

In [ ]:
traceSynthetic = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(1)

traceSynthetic[1] = scatter(
    x=syntheticDFfiltered[:KdWeight],
    y=syntheticDFfiltered[:POC_noOutliers],
    marker_color = length.(syntheticDFfiltered[:GenesKD]),
    mode="markers"
)
layoutSynthetic = Layout(xaxis_title="<b>Sum of disrupted weights</b>", yaxis_title = "<b>POC</b>")

plot(traceSynthetic, layoutSynthetic)

In [ ]:
# Initialize all weights to 0
syntheticDFfiltered[:RemWeightKD] = 0
syntheticDFfiltered[:RemWeightKD] = Float64.(syntheticDFfiltered[:RemWeightKD])

# For each sample, compute the sum of the disrupted interactions
for (sampleIndex, sample) in enumerate(syntheticDFfiltered[:GenesKD])
    sampleIndices = [unitDict[gene] for gene in sample]
    for u in 1:M if !(u in sampleIndices)
        for v in (u+1):M if !(v in sampleIndices)
            tupleKey = (u,v)
            if tupleKey in keys(averageEdges)
                syntheticDFfiltered[sampleIndex,:RemWeightKD] += averageEdges[tupleKey]
            end
        end end
    end end
end

In [ ]:
traceSynthetic = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(1)

traceSynthetic[1] = scatter(
    x=syntheticDFfiltered[:RemWeightKD],
    y=syntheticDFfiltered[:POC_noOutliers],
    marker_color = length.(syntheticDFfiltered[:GenesKD]),
    mode="markers"
)
layoutSynthetic = Layout(xaxis_title="<b>Remaining weights</b>", yaxis_title = "<b>POC</b>")

plot(traceSynthetic, layoutSynthetic)

In [ ]:
println(cor(syntheticDFfiltered[:RemWeightKD],syntheticDFfiltered[:deltaPOC_noOutliers]))
println(cor(syntheticDFfiltered[:RemWeightKD],syntheticDFfiltered[:POC_noOutliers]))
println(cor(syntheticDFfiltered[:RemWeightKD],syntheticDFfiltered[:KdWeight]))

In [ ]:
CSV.write("syntheticDataWithWeights.tsv", syntheticDFfiltered; delim = '\t')

In [ ]:
traceSynthetic = Array{PlotlyBase.GenericTrace{Dict{Symbol,Any}}}(1)

traceSynthetic[1] = scatter(
    x=syntheticDF[.!ismissing.(syntheticDF[:TotalVolumeTransferred]),:TotalVolumeTransferred],
    y=syntheticDF[.!ismissing.(syntheticDF[:TotalVolumeTransferred]),:deltaPOC_noOutliers],
    mode="markers"
)
layoutSynthetic = Layout(xaxis_title="<b>Volume transferred</b>", yaxis_title = "<b>deltaPOC</b>")

plot(traceSynthetic, layoutSynthetic)

In [ ]:
find(ismissing.(syntheticDF[:TotalVolumeTransferred]))

In [ ]:
readdir()